In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import folium as fl
from folium.plugins import HeatMap

import plotly.express as px

import glob
import ast
import re

from processing.parsing import *

In [2]:
def read_csv(path: str) -> pd.DataFrame:
    with open(path, encoding = 'utf-8') as file:
        data = file.readlines()
        
    pattern_dict = r'\{([^}]+)\}' #patrón para detectar diccionarios
    pattern_list = r'\[([^]]+)\]' #patrón para detectar listas

    df = pd.DataFrame(columns=['price', 'title_location', 'lat', 'lng', 'province', 'last_updated', 'agency', 'characteristics', 'meta_data'])
    
    for row in data[1:]:
        new_row = {}

        match = re.findall(pattern_dict, row) # buscamos cualquier cosa que parezca un diccionario
        if len(match) == 0:
            new_row['meta_data'] = np.nan
            row = row[:- 3]
        else:
            new_row['meta_data'] = ast.literal_eval('{' + match[-1] + '}') # parseamos el diccionario

            row = row[:- 4 -len(match[-1])] # borramos la representación del diccionario de meta_data
        row = row.split(',') # separamos por coma
        
        new_row['last_updated'] = row.pop() # sacamos la columna last_updated
        new_row['agency'] = row.pop() # sacamos la columna agency

        row = ','.join(row) # juntamos con comas

        match = re.findall(pattern_list, row) # buscamos cualquier cosa que parezca una lista

        try:
            if len(match) == 0:
                new_row['characteristics'] = np.nan
            else:
                new_row['characteristics'] = ast.literal_eval('[' + match[-1] + ']') # parseamos la lista
        except:
            new_row['characteristics'] = np.nan

        try:
            row = row[:- 4 -len(match[0])] # borramos la representación de la lista
        except:
            row = row[:-3]

        row = row.split(',') # separamos por coma

        new_row['lng'] = row.pop() # sacamos columna de longutud
        new_row['lat'] = row.pop() # sacamos columna de latitud
        new_row['price'] = row.pop(0) # sacamos la columna de precio

        new_row['title_location'] = ','.join(row) # sacamos titulo y calle (hay comas molestando, no sé como separar bien las dos columnas)
        
        new_row['province'] = path

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    return df

def tryNan(x):
    if pd.isna(x):
        return np.nan
    if 'consultar' in x:
        return np.nan
    
    try:
        return int(''.join(x[:-2].split('.')))
    except:
        return np.nan
    
def tryFloat(x):
    try:
        return float(x)
    except:
        return np.nan

In [3]:
df = pd.concat([read_csv(x) for x in glob.glob('../scraping/data/*.csv')])

df['price'] = df['price'].apply(tryNan)
df['lat'] = df['lat'].apply(tryFloat)
df['lng'] = df['lng'].apply(tryFloat)

In [105]:
# df.to_csv('sample.csv', index = False)

df_ali = pd.read_csv('../data/alicante.csv')
df_bar = pd.read_csv('../data/barcelona.csv')
df_mad = pd.read_csv('../data/madrid.csv')
df_mal = pd.read_csv('../data/malaga.csv')
df_val = pd.read_csv('../data/valencia.csv')

In [106]:
df = pd.concat([df_ali, df_bar])

In [107]:
# Input de timestamp
# df_mal
# df_val

Pasos para organizar la data:
- Unir 'title' y 'location'
- Añadir el timestamp a 'numeric_data'

Orden de columnas:
- price, title, location, province, lat, lng, characteristics, agency, updated, timestamp, id_1, id_2

In [108]:
df

,price,title,province,location,lat,lng,characteristics,agency,updated,numeric_data
0,75.000 €,Piso en venta en Biar,alicante,Biar,38.628784,-0.761397,"['Superficie construida : 76 m²', 'Superficie ...",Inmuebles de Topbrokers,1.673392e+09,"['1699186637712631', '22524900237', '100500']"
1,56.000 €,Piso en venta en Carrer de Severo Ochoa,alicante,Carrer de Severo Ochoa. Sax,38.535113,-0.821114,"['Superficie construida : 114 m²', 'Superficie...",Inmuebles de CICLOACTIVOS S.L.,1.673997e+09,"['1699186642826594', '945856907971029', '109700']"
2,39.800 €,"Piso en venta en Calle de Joaquín María López, 23",alicante,"Calle de Joaquín María López, 23. Villena",38.631641,-0.860810,"['Superficie construida : 82 m²', 'Superficie ...",Inmuebles de ALTAMIRA,1.673133e+09,"['1699186647038339', '25053088111', '103600']"
3,119.500 €,"Piso en venta en Carrer de Artal de Alagón, 11",alicante,"Carrer de Artal de Alagón, 11. Sax",38.537183,-0.817027,"['Superficie construida : 220 m²', 'Superficie...",Inmuebles de Grupo Ideas Sax,1.674170e+09,"['1699186650805199', '933640558', '184300']"
4,103.000 €,Casa en venta en Biar,alicante,Biar,38.631503,-0.765994,"['Superficie construida : 170 m²', 'Superficie...",Inmuebles de Quo Real Estate,1.672701e+09,"['1699186655050266', '35027955014', '100500']"
...,...,...,...,...,...,...,...,...,...,...
27476,275.000 €,Casa adosada en venta en Vallès Oriental,barcelona,Parets del Vallès,41.556418,2.224938,"['Superficie construida : 181 m²', 'Superficie...",Inmuebles de Engel & Völkers Barcelona,1.673997e+09,"['1697946884084809', '38334918184', '101800']"
27477,330.000 €,"Casa en venta en Calle Ps Catalunya De, nº 2",barcelona,"Calle Ps Catalunya De, nº 2. Aiguafreda",41.768390,2.251378,"['Superficie construida : 187 m²', 'Superficie...",Inmuebles de ESPAI HABITAT,1.673997e+09,"['1697946890074617', '34237375728', '109300']"
27478,570.000 €,Casa en venta en Vallès Oriental,barcelona,Corró d'Avall (Les Franqueses del Vallès),41.638051,2.312595,"['Superficie construida : 530 m²', 'Superficie...",Inmuebles de Engel & Völkers Barcelona,1.673651e+09,"['1697946895652892', '37515155048', '101800']"
27479,279.800 €,Piso en venta en Caldes de Montbui,barcelona,Caldes de Montbui,41.626662,2.170873,"['Superficie construida : 122 m²', 'Superficie...",Inmuebles de AGC FINQUES,1.674083e+09,"['1697946901545553', '37581966089', '170800']"


In [109]:
# Procesamiento Madrid
df_mad['province'] = ['madrid' for x in range(df_mad.shape[0])]
df_mad['timestamp'] = [1696687200 for x in range(df_mad.shape[0])]
df_mad['id_1'] = [x for x in range(df_mad.shape[0])]
df_mad['id_2'] = [x for x in range(df_mad.shape[0])]
df_mad.drop('Unnamed: 0', axis = 1, inplace = True)

# Procesamiento Valencia
df_val['province'] = df_val['province'].apply(lambda x: 'valencia')
df_val['timestamp'] = [1698233325 for x in range(df_val.shape[0])]
df_val['id_1'] = [x[-2] for x in df_val['numeric_data'].apply(ast.literal_eval)]
df_val['id_2'] = [x[-1] for x in df_val['numeric_data'].apply(ast.literal_eval)]

# Procesamiento Malaga
df_mal['timestamp'] = [1698266916 for x in range(df_mal.shape[0])]
df_mal['id_1'] = [x[-2] for x in df_mal['numeric_data'].apply(ast.literal_eval)]
df_mal['id_2'] = [x[-1] for x in df_mal['numeric_data'].apply(ast.literal_eval)]

# Procesamiento Alicante / Barcelona
df['timestamp'] = [x[0] for x in df['numeric_data'].apply(ast.literal_eval)]
df['id_1'] = [x[-2] for x in df['numeric_data'].apply(ast.literal_eval)]
df['id_2'] = [x[-1] for x in df['numeric_data'].apply(ast.literal_eval)]

In [110]:
df = pd.concat([df, df_mal, df_val, df_mad]).drop('numeric_data', axis = 1)
df['characteristics'] = df['characteristics'].apply(ast.literal_eval)
df['timestamp'] = df['timestamp'].astype(str)
df['id_1'] = df['id_1'].astype(str)
df['id_2'] = df['id_2'].astype(str)
df

,price,title,province,location,lat,lng,characteristics,agency,updated,timestamp,id_1,id_2
0,75.000 €,Piso en venta en Biar,alicante,Biar,38.628784,-0.761397,"[Superficie construida : 76 m², Superficie úti...",Inmuebles de Topbrokers,1.673392e+09,1699186637712631,22524900237,100500
1,56.000 €,Piso en venta en Carrer de Severo Ochoa,alicante,Carrer de Severo Ochoa. Sax,38.535113,-0.821114,"[Superficie construida : 114 m², Superficie út...",Inmuebles de CICLOACTIVOS S.L.,1.673997e+09,1699186642826594,945856907971029,109700
2,39.800 €,"Piso en venta en Calle de Joaquín María López, 23",alicante,"Calle de Joaquín María López, 23. Villena",38.631641,-0.860810,"[Superficie construida : 82 m², Superficie úti...",Inmuebles de ALTAMIRA,1.673133e+09,1699186647038339,25053088111,103600
3,119.500 €,"Piso en venta en Carrer de Artal de Alagón, 11",alicante,"Carrer de Artal de Alagón, 11. Sax",38.537183,-0.817027,"[Superficie construida : 220 m², Superficie út...",Inmuebles de Grupo Ideas Sax,1.674170e+09,1699186650805199,933640558,184300
4,103.000 €,Casa en venta en Biar,alicante,Biar,38.631503,-0.765994,"[Superficie construida : 170 m², Superficie út...",Inmuebles de Quo Real Estate,1.672701e+09,1699186655050266,35027955014,100500
...,...,...,...,...,...,...,...,...,...,...,...,...
15841,135.000 €,Piso en venta en Calle de la Circunvalación,madrid,Calle de la Circunvalación. Zona Suroeste (Tor...,40.454606,-3.455234,"[Superficie construida : 67 m², Superficie úti...",Inmuebles de ANGEL & RICARDO INMOBILIARIA,1.672960e+09,1696687200,15841,15841
15842,210.000 €,Piso en venta en Calle de Urano,madrid,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),40.441928,-3.473036,"[Superficie construida : 79 m², Superficie úti...",Inmuebles de Look & Find Retiro,1.672615e+09,1696687200,15842,15842
15843,324.600 €,Piso en venta en Fresnos,madrid,Zona Suroeste (Torrejón de Ardoz),40.448403,-3.470899,"[Superficie construida : 140 m², Habitaciones ...",Inmuebles de Cassas.es asesoria inmobiliaria,1.672874e+09,1696687200,15843,15843
15844,249.900 €,Piso en venta en Calle de Cibeles,madrid,Calle de Cibeles. Zona Suroeste (Torrejón de A...,40.444707,-3.473520,"[Superficie construida : 106 m², Superficie út...",Inmuebles de AS Fresnos Inmobiliaria,1.672874e+09,1696687200,15844,15844


In [111]:
df.to_parquet('processed_data/la_yaya.parquet')

## Ahora toca procesar lo nuevo

In [5]:
fig = px.bar([df[x].dropna().count() for x in df.columns],
             y = df.columns,
             x = [df[x].dropna().count() for x in df.columns])

fig.update_layout(height = 1000)

In [33]:
# Lista de columnas a revisar para decidir cómo tratar la información para entrenar el modelo en el futuro
scratched_features_col = ['Cocina', 'Cocina equipada', 'Superficie solar', 'Sistema de seguridad', 'Portero automático',
                      'Aire acondicionado', 'Puerta blindada', 'Amueblado', 'Trastero', 'Calefacción', 'Gas',
                      'Tipo de casa', 'Piscina', 'Vidrios dobles', 'Planta', 'Tipo suelo', 'Balcón', 'Orientación',
                      'Armarios empotrados', 'Comedor']

# Cocina -> hacerla categórica (americana, independiente, amueblada, sin amueblar)
# Cocina equipada -> revisarla con 'Cocina'
# Superficie solar -> Seguramente sea interesante solo para chalets
# Sistema de seguridad -> hacerla categórica (conserje, alarma, videovigilancia, etc.) mezclándola con 'Portero automático'
# Portero automático -> relacionar con 'Sistema de seguridad'
# Aire acondicionado -> hacerla categórica numérica
# Puerta blindada -> hacerla binaria
# Amueblado -> hacerla categórica (sí, no, parcial)
# Trastero -> hacerla categórica (sí, no, comunitario)
# Calefacción -> hacerla categórica (gas natural, butano, eléctrico, frío-calor, chimenea, no)
# Gas -> hacerla categórica (revisar Calefacción)
# Tipo de casa -> hacerla categórica ('Adosada', 'Pareada', ' Rústica', ' Unifamiliar', 'otro)
# Piscina -> hacerla categórica (privada, comunitaria, opcional-comunitaria, no, etc)
# Vidrios dobles -> hacerla categórica (corredera, oscilobatiente, abrir-cerrar, doble aluminio, no)
# Planta -> hacerla categórica numérica (-2, -1, 0, 1, 2, etc)
# Tipo suelo -> hacerla binaria (frío, caliente)
# Balcón -> hacerla categórica numérica
# Armarios empotrados -> hacerla binaria
# Comedor -> hacerla categórica

In [34]:
features_col = ['price', 'title', 'lat', 'lng', 'location', 'Habitaciones', 'Superficie construida',
                'Jardín', 'Gastos de comunidad', 'Antigüedad', 'Baños', 'Superficie útil', 'Ascensor',
                'Garaje', 'Conservación', 'agency', 'updated']

# Garaje -> hacer varias columnas columnas (tien o no tiene, cantidad, metros cuadrados)
# location -> sacar columnas (tipo de vía, locación, etc)
# title -> revisar con columna 'Tipo de casa' para obtener una columna con tipo de vivienda

In [35]:
df_features = df[features_col]

KeyError: "['title', 'location', 'Habitaciones', 'Superficie construida', 'Jardín', 'Gastos de comunidad', 'Antigüedad', 'Baños', 'Superficie útil', 'Ascensor', 'Garaje', 'Conservación', 'updated'] not in index"

In [36]:
# Sería interesante poner una columna que tenga la fecha actual? Para que el modelo compare con la fecha de publicación

In [37]:
df_ = getType(df_features)


NameError: name 'df_features' is not defined

In [ ]:
df_ = getStreetType(df_)

c:\Users\cuicuidev\Desktop\pisoscom\ml\processing\parsing.py:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
streets = df_['street'].dropna().unique()

In [ ]:
unique_streets = []

for street in streets:
    for s in street.split():
        if s == 'c' or s == 'av':
            continue
        unique_streets.append(s)

unique_streets = list(set(unique_streets))

In [ ]:
unique_streets_en = {'calle' : 'street', 'bulevar' : 'boulevard', 'plaza' : 'square', 'autovía' : 'highway', 'carretera' : 'road', 'parque' : 'park', 'paseo' : 'walk', 'avenida' : 'avenue'}
unique_streets

['bulevar',
 'paseo',
 'carretera',
 'parque',
 'calle',
 'autovia',
 'avenida',
 'plaza']

In [ ]:
for street in unique_streets:
    df_[street] = df_['street'].apply(lambda x: 0 if pd.isna(x) else sum([1 for _ in x.split() if 'calle' in x]))

df_ = df_.drop('street', axis = 1)

C:\Users\cuicuidev\AppData\Local\Temp\ipykernel_13204\917673754.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\cuicuidev\AppData\Local\Temp\ipykernel_13204\917673754.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\cuicuidev\AppData\Local\Temp\ipykernel_13204\917673754.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [ ]:
df_

,price,title,lat,lng,location,Habitaciones,Superficie construida,Jardín,Gastos de comunidad,Antigüedad,...,updated,type,bulevar,paseo,carretera,parque,calle,autovia,avenida,plaza
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,40.593445,-4.145386,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,1.0,63 m²,Comunitario,Entre 40 y 60€,Entre 10 y 20 años,...,1.674861e+09,Apartamento,0,0,0,0,0,0,0,0
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...",40.402079,-3.702151,"Calle de Ercilla, 20, cerca de Calle del Labra...",NaN,25 m²,NaN,NaN,Más de 50 años,...,1.674947e+09,Apartamento,2,2,2,2,2,2,2,2
2,169000.0,Apartamento en venta en Ajalvir,40.534457,-3.479415,Ajalvir,1.0,70 m²,NaN,NaN,NaN,...,1.673392e+09,Apartamento,0,0,0,0,0,0,0,0
3,174000.0,Apartamento en venta en Alcorcón,40.344358,-3.825283,Casco Antiguo (Alcorcón),1.0,60 m²,NaN,NaN,NaN,...,1.672960e+09,Apartamento,0,0,0,0,0,0,0,0
4,130000.0,Apartamento en venta en Alcorcón,40.347096,-3.827826,Casco Antiguo (Alcorcón),2.0,60 m²,NaN,Entre 60 y 80€,NaN,...,1.672615e+09,Apartamento,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15841,135000.0,Piso en venta en Calle de la Circunvalación,40.454606,-3.455234,Calle de la Circunvalación. Zona Suroeste (Tor...,3.0,67 m²,NaN,NaN,NaN,...,1.672960e+09,Piso,1,1,1,1,1,1,1,1
15842,210000.0,Piso en venta en Calle de Urano,40.441928,-3.473036,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),1.0,79 m²,NaN,Entre 80 y 100€,NaN,...,1.672615e+09,Piso,1,1,1,1,1,1,1,1
15843,324600.0,Piso en venta en Fresnos,40.448403,-3.470899,Zona Suroeste (Torrejón de Ardoz),4.0,140 m²,NaN,NaN,NaN,...,1.672874e+09,Piso,0,0,0,0,0,0,0,0
15844,249900.0,Piso en venta en Calle de Cibeles,40.444707,-3.473520,Calle de Cibeles. Zona Suroeste (Torrejón de A...,3.0,106 m²,NaN,Entre 80 y 100€,Entre 20 y 30 años,...,1.672874e+09,Piso,1,1,1,1,1,1,1,1


In [ ]:
df_ = df_.drop('title', axis = 1)

In [ ]:
df_ = df_.drop('location', axis = 1)

# Parsing de 'agency'

In [ ]:
agency_col = df_['agency'].unique()

agencies = []
for agency in agency_col:
        
    if pd.isna(agency):
        continue
    if 'inmuebles' in agency.lower():
        agencies.append(agency)

len(agencies)

1325

In [ ]:
df_['publisher'] = df_['agency'].apply(lambda x: 'particular' if x not in agencies else 'inmobiliaria')

In [ ]:
df_ = df_.drop('agency', axis = 1)

# Parsing de 'Gastos de comunidad'

In [70]:
len(df_['Gastos de comunidad'].unique())

# CASOS
# 'Entre X y Y€'
# x,y
# x
# Más de x€
# x€
# x,y€
# nan, Gastos de comunidad, anual, trimestral/trimestre, 0, No, 
# x + y

360

In [71]:
# De momento quitamos esa columna
df_ = df_.drop('Gastos de comunidad', axis = 1)

# Parsing de 'Antigüedad'

In [72]:
df_['Antigüedad'].unique()

array([' Entre 10 y 20 años', ' Más de 50 años', nan,
       ' Entre 30 y 50 años', ' Entre 20 y 30 años', ' Entre 5 y 10 años',
       ' Menos de 5 años'], dtype=object)

In [73]:
antiguedad_col = df_['Antigüedad'].unique()
antiguedades = {' Menos de 5 años': 0,
                ' Entre 5 y 10 años' : 5,
                ' Entre 10 y 20 años' : 10,
                ' Entre 20 y 30 años' : 20,
                ' Entre 30 y 50 años' : 30,
                ' Más de 50 años': 50,}

In [74]:
df_['age'] = df_['Antigüedad'].replace(antiguedades)

In [75]:
df_ = df_.drop('Antigüedad', axis = 1)

# Parsing de 'Garaje'

In [76]:
df_['garage'] = df_['Garaje'].apply(lambda x: 'no' if pd.isna(x) else 'yes')

In [77]:
df_ = df_.drop('Garaje', axis = 1)

# Parsing de 'Ascensor'

In [78]:
# algunos pone 'no tiene'
df_['lift'] = df_['Ascensor'].apply(lambda x: 'no' if pd.isna(x) else 'yes')

In [79]:
df_ = df_.drop('Ascensor', axis = 1)

# Parsing de 'Superficie construida'

In [80]:
df_['surface'] = df_['Superficie construida'].apply(lambda x: np.nan if pd.isna(x) else int(''.join([_ for _ in x[:-3] if _.isnumeric()])))

In [81]:
df_ = df_.drop('Superficie construida', axis = 1)

# Parsing de 'Superficie útil'

In [82]:
df_['net_surface'] = df_['Superficie útil'].apply(lambda x: np.nan if pd.isna(x) else int(''.join([_ for _ in x[:-3] if _.isnumeric()])))

In [83]:
df_ = df_.drop('Superficie útil', axis = 1)


# Parsing de 'Jardín'

In [84]:
df_['garden'] = df_['Jardín'].apply(lambda x: 'no' if pd.isna(x) else 'yes')

In [85]:
df_ = df_.drop('Jardín', axis = 1)

# Renaming de columnas en español

In [86]:
df_['rooms'] = df_['Habitaciones']
df_['condition'] = df_['Conservación']
df_['bathrooms'] = df_['Baños']

df_ = df_.drop(['Habitaciones', 'Baños', 'Conservación'], axis = 1)

# Save

In [87]:
#df_.to_csv('../data/madrid_preprocessed.csv', index = False)

# Read

In [38]:
#df_ = pd.read_csv('../data/madrid_preprocessed.csv')
df_ = df

In [39]:
# Distribución de los precios para encontrar outliers
fig = px.histogram(data_frame=df_, x='price')
fig.show()

In [40]:
# Distribución de la cantidad de baños
fig = px.histogram(data_frame=df_, x='bathrooms')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: bathrooms

In [41]:
# Distribución de la cantidad de baños
fig = px.histogram(data_frame=df_, x='surface')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: surface

In [42]:
# Distribución del número de habitaciones para encontrar outliers
fig = px.histogram(data_frame=df_, x='rooms')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: rooms

In [ ]:
# Distribución de la condición de las viviendas
fig = px.histogram(data_frame=df_, x='condition')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: condition

In [43]:
# Distribución del tipo de vivienda
fig = px.histogram(data_frame=df_, x='type')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: type

In [44]:
# Distribución de los precios por número de habitaciones y por publisher
fig = px.box(df_.dropna(), x='rooms', y='price', facet_col='publisher')
fig.update_layout(title='Rooms vs Price by the publisher', xaxis_title='Nº of rooms', yaxis_title='Prices')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: rooms

In [45]:
# Cantidad de publicaciones hechas por inmobiliarias vs particulares
fig = px.pie(df_, names='publisher', values=df_.index)
fig.update_traces(textinfo='percent+label')
fig.update_layout(title='Amount of advertisements by the publisher')
fig.show()

ValueError: Value of 'names' is not the name of a column in 'data_frame'. Expected one of ['price', 'lat', 'lng', 'last_updated', 'agency', 'characteristics', 'meta_data'] but received: publisher

In [46]:
# Mapa de calor
heatmap_ = fl.Map(location=[36.9592884,-6.5695077], zoom_start=5)
HeatMap(df_[['lat', 'lng', 'price']].dropna()).add_to(heatmap_)
heatmap_

# heatmap_.save("heatmap_by_prices.html")

ValueError: Location values cannot contain NaNs.

In [98]:
# # Mapa con todas las ubicaciones de las viviendas (quitando nans)
# # Cuesta ejecutarlo
# map = fl.Map(location=[36.9592884,-6.5695077], zoom_start=5)
# for loc in [(x, y) for x, y in zip(df_['lat'].dropna(), df_['lng'].dropna())]:
#     fl.Marker(loc).add_to(map)
# # map

# map.save("locations_map.html")

In [47]:
df_[['price', 'lat', 'lng']]

,price,lat,lng
0,42.000,36.959564,-2.57303
1,27.000,36.9948,-2.6807
2,25.000,36.9752872,-2.962034
3,160.000,37.007212211,-2.74079746
4,525.000,36.9875,-2.7388
...,...,...,...
6590,150.000,37.3430290222168,-2.0861299037933
6591,NaN,37.35192,-2.193317
6592,125.000,37.3644112,-1.677711
6593,40.000,37.3062,-2.3118
